In [1]:
### --- maDLC2 Web Application with voilà ---
# --- 1. Import librairies ---
# GUI Jupyter
import ipywidgets as widgets
from ipywidgets import IntProgress, Layout, GridBox, HTML
from IPython.display import display, clear_output
import base64
import webbrowser
import os

# DeepLabCut
from numba import cuda
## GUI don't work on the cloud, so we supress them:
os.environ["DLClight"]="True"

terminal = widgets.Output()

with terminal:
    import deeplabcut

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import os
import fnmatch
from datetime import datetime
from zipfile import ZipFile
from os.path import basename

# set the plots to display in the Jupyter notebook
# %matplotlib inline

# change plotting colors per client request
plt.style.use('ggplot')

# Increase default figure and font sizes for easier viewing.
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 14

# print('Done!')

In [3]:
# --- 2. Create widgets --- 

# Image Widget
file = open("/home/ubuntu/Files/Hands.png", "rb")
image = file.read()
image_hands = widgets.Image(
                    value=image,
                    format='jpg',
                    width='300',
                )

file = open("/home/ubuntu/Files/DLC_logo.png", "rb")
image = file.read()
image_dlc = widgets.Image(
                    value=image,
                    format='jpg',
                    width='150',
                )

file = open("/home/ubuntu/Files/WTScale.png", "rb")
image = file.read()
image_wtscale = widgets.Image(
                    value=image,
                    format='jpg',
                    width='150',
                )

# File uploader
uploader = widgets.FileUpload(
    accept='.mp4',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

# Buttons
button_send = widgets.Button(
                description='Launch full analysis',
                disabled=True,
                style={'description_width': 'initial'},
                layout=Layout(height='5rem', align_self='center')
            )

# button_download = widgets.Button(
#                 description='Download results (plots, raw data, output video)',
#                 disabled=False,
#                 style={'description_width': 'initial'},
#                 layout=Layout(height='5rem', align_self='center')
#             )

radio_tracking = widgets.RadioButtons(
    options=['box', 'skeleton', 'ellipse'],
    value='ellipse', # Defaults to 'ellipse'
#    layout={'width': 'max-content'}, # If the items' names are long
    layout=Layout(width='auto', align_self='center'),
    description='Tracking type',
    disabled=False
)

# Labels
label_hands = widgets.Label(
                    value='Tracking example',
                    style={'description_width': 'initial'}
                )

label_dlc = widgets.Label(
                    value='DeepLabCut version: ' + deeplabcut.__version__,
                    style={'description_width': 'initial'}
                )

dlcP_headline = widgets.Label(
                    value='Video analysis status',
                    style={'description_width': 'initial'}
                )
qP_headline = widgets.Label(
                    value='Quantification status',
                    style={'description_width': 'initial'}
                )
label_terminal = widgets.Label(
                    value='You can see real time progression in the terminal',
                    style={'description_width': 'initial'}
                )

# Others
# download_button = widgets.ToggleButton()
dlcProgress = IntProgress(min=0, max=100) # instantiate the bar
qProgress = IntProgress(min=0, max=100) # instantiate the bar
output = widgets.Output()
# download_output = widgets.Output()
result_output = widgets.Output()

In [4]:
# --- 3. Widgets methods --- 
today = datetime.today().strftime('%Y-%m-%d')
fileName='MM_DLC-Analysis_'+today+'.zip'

# OBSERVE upload
def on_upload_change(change):
    # So, to get the filename, and get the file content:
    uploaded_filename = next(iter(uploader.value))
    content = uploader.value[uploaded_filename]['content']
    with open(os.path.join('/home/ubuntu/Files/Inference','Video.mp4'), 'wb') as f: f.write(content)
        
    # Disable the upload, enable analysis   
    uploader.disabled = True
    button_send.disabled = False
    
    with output:
        print('- Video uploaded, click on "Launch full analysis" to start the process !')    

uploader.observe(on_upload_change, names='_counter')

def on_button_analyze(event):
    with output:
        print(radio_tracking.value)
        print("- Pose extraction started")
    dlc_full()
    with output:
        print("- Analysis over. Proceeding with quantification.")
    full_quantification() 
    with output:
        print("- Quantification is over.")
    fileName = zipAll()
#     download_link.layout.visibility='visible'
    download_link.layout.display='inline-block'
    with output:
        print("- Files ready to download")
    
def zipAll():
    with terminal:
        if tracktype == 'box':
            tType = 'bx'
        elif tracktype == 'skeleton':
            tType = 'sk'
        elif tracktype == 'ellipse':
            tType = 'el'
            
        path='Files/Inference/'
        zipObj = ZipFile(fileName, 'w')

        zipObj.write('VideoDLC_resnet50_maDLC_MM_handsJul8shuffle1_20000_'+tType+'_filtered_bp_labeled.mp4')
        zipObj.write('VideoDLC_resnet50_maDLC_MM_handsJul8shuffle1_20000_'+tType+'_filtered.csv')

        zip_folder('plots', zipObj)
        zip_folder('plot-poses/Video', zipObj)
        zip_folder('plots-normalized', zipObj)
        zip_folder('plots-xcross', zipObj)
        zipObj.close()
        print(fileName)
        return fileName
        
# Method to zip an entire folder
def zip_folder(path, zipO):
    files = fnmatch.filter(os.listdir(path), "*.png")
    for f in files:
        zipO.write(path+'/'+f) 

def what_traits_radio(value):
    tracktype=value['new']
    with terminal:
        print('Track type selected : ' + value['new'])
              
        
radio_tracking.observe(what_traits_radio, names = 'value')

# Methods
button_send.on_click(on_button_analyze)
# button_download.on_click(on_button_download)

In [5]:
# Show results
hbox_plots = widgets.HBox([image_dlc], layout=Layout(display="flex", width="100%", justify_content='center'))

def getTemplateGrid():
    return GridBox(children=[],
        layout=Layout(
            width='100%',
            grid_template_columns='50% 50%',
            grid_template_rows='30% 30% 30%',
        )
    )

def appendImages(folder):
    path = folder
    files = fnmatch.filter(os.listdir(path), "*.png")
    
    tempGB = getTemplateGrid()

    for f in files :
        tmpFile = open(folder+'/'+f, "rb")
        tmpImg = tmpFile.read()
        newWidget = widgets.Image(
                        value=tmpImg,
                        format='jpg',
#                         width='auto',
                    )
        tempGB.children+=(newWidget,)
        
    return tempGB

gb_1 = widgets.Output()
gb_2 = widgets.Output()
gb_3 = widgets.Output()


In [31]:
# --- 4. Widgets assigments --- 


#HTML widgets
text_0 = widgets.HTML(value="<h1 style=""text-align:center"">Detect and quantify mirror movements in videos of children with cerebral palsy using DeepLabCut</h1>")
text_1= widgets.HTML(value="<h2>Upload your file</h2>")
text_2= widgets.HTML(value="<h2>Instructions for use</h2>")
text_3= widgets.HTML(value='''<p>Please read and follow carefully <a style="color:blue; text-decoration: underline" href="https://www.google.ch/" target="_blank">the documentation</a> to understand how to correctly take the video. Many factors (stability, framing, angle) can influence detection and quantification.</p>''')
text_4= widgets.HTML(value="<p><b>Steps</b> : <ol style=""margin:0px;""><li>Upload a video</li><li>Launch the process</li><li>Watch/Get the results !</li></ol></p>")
text_5= widgets.HTML(value="<p><b>Video requirements</b> : <ul style=""margin:0px;""><li>Only MPEG-4 (.mp4) extension</li><li>No longer than 10 seconds !</li></ul></p>")
text_6= widgets.HTML(value='''<p><b>Warranty</b> : </br>This application does not come with any warranty. Graphics and measures depend on how the video was taken according to <a style="color:blue; text-decoration: underline" href="https://www.google.ch/" target="_blank">the documentation</a>. Even if all the conditions are met for a good quantification, the user must always keep a critical eye on the results. Indeed, this is an experimental application and the graphs serve as a tool to understand how the movements were tracked. It is important to check the output video of the software to ensure that the motion tracking has been performed correctly.</p>''')

text_7= widgets.HTML(value="<h3>Video analysis status</h3>")
text_8= widgets.HTML(value="<h3>Quantification status</h3>")

text_9= widgets.HTML(value="<p>Experimental application ©2021 Brice Berclaz & HES-SO Valais-Wallis.</p>")
text_10= widgets.HTML(value="<p><i>No copying or reproduction without the agreement of the course leader, the professor in charge and the author.</i></p>")

# fileName='MM_DLC-Analysis_2021-07-30.zip'
text_download= widgets.HTML(value="<p>Download button will be displayed automatically once the results are available.</p>")
download_link = widgets.HTML(value=f'''<a href="https://jupyter-conda-e865db17.proxy.gnsiscld.co/files/Files/Inference/{fileName}" target="_blank">
<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">Download files</button>
</a>''')
# download_link.layout.visibility='hidden'
download_link.layout.display = "none"


# text_pearson= widgets.HTML(value="<p>Levels : 0 <= 0.35 / 1 : >= 0,5075 / 2 : >= 0,665 / 3 : >= 0,8225</p> / 4 : >= 0,96")

vbox_results = widgets.VBox([image_wtscale, result_output, text_download, download_link], layout=Layout(width="auto"))



# Accordion
accordion = widgets.Accordion(description='Plots', children=[gb_1])
accordion2 = widgets.Accordion(description='Normalized plots', children=[gb_2])
accordion3 = widgets.Accordion(description='Cross-correlation', children=[gb_3])
accordion4 = widgets.Accordion(description='Terminal', children=[terminal])
accordion5 = widgets.Accordion(description='Results', children=[vbox_results]) #button_download,
accordion.set_title(0, 'Plots')
accordion2.set_title(0, 'Normalized plots')
accordion3.set_title(0, 'Cross-correlation')
accordion4.set_title(0, 'Terminal')
accordion5.set_title(0, 'Results')
accordion2.selected_index = None
accordion3.selected_index = None
# accordion4.selected_index = None
accordion5.selected_index = None

# Layout

vbox_manage = widgets.VBox([text_1, uploader, radio_tracking], layout=Layout(width="auto"))
vbox_button = widgets.VBox([button_send, label_terminal],  layout=Layout(display="flex", justify_content='center', width="auto")) #justify_content='center', align_self='center',
vbox_dlc = widgets.VBox([image_dlc, label_dlc], layout=Layout(width="auto", display="flex", align_items='center'))

vbox_status = widgets.VBox([image_hands, label_hands, text_7, dlcProgress, text_8, qProgress, output],  layout=Layout(width="17%", align_items='center'))
# vbox_result = widgets.VBox([button_send])
# vbox_manage = widgets.VBox([text_1, uploader, button_send])

# vbox_options = widgets.VBox([radio_tracking], layout=Layout(display="flex", width="50%", justify_content='center', align_self='center', align_items='center'))

# hbox_tool = widgets.HBox([vbox_manage, vbox_options])
hbox_tool = widgets.HBox([vbox_manage, vbox_button, vbox_dlc], layout=Layout(display="flex", width="100%", justify_content='space-around'))#, #justify_content='center', align_self='center', align_items='center'))
vbox_center = widgets.VBox([hbox_tool, accordion, accordion2, accordion3, accordion4, accordion5], layout=Layout(width="55%")) #, justify_content='center', align_self='center', align_items='center'))
vbox_instructions = widgets.VBox([text_2, text_3, text_4, text_5, text_6], layout=Layout(width="20%"))
vbox_footer = widgets.VBox([text_9, text_10], layout=Layout(display="flex", width="auto%", align_items='center'))

hbox_header = widgets.HBox([text_0], layout=Layout(display="flex", width="100%", justify_content='center'))
hbox_body = widgets.HBox([vbox_status, vbox_center, vbox_instructions], layout=Layout(display="flex", width="100%", justify_content='space-between'))
hbox_footer = widgets.HBox([vbox_footer], layout=Layout(display="flex", width="100%", justify_content='center'))

In [7]:
# --- 5. Assign layout ---
page = widgets.VBox([hbox_header, hbox_body, hbox_footer])
display(page)

In [8]:
# --- 6. DeepLabCut methods for inference ---
VideoType = 'mp4'
shuffle=1
tracktype= 'ellipse' #box, skeleton, ellipse

path_config_file = '/home/ubuntu/Files/maDLC_MM_hands-Brice-2021-07-08/config.yaml'
Specific_videofile = '/home/ubuntu/Files/Inference/Video.mp4'
videofile_path = ['/home/ubuntu/Files/Inference']

def analyze_video():
    with terminal:
        scorername = deeplabcut.analyze_videos(path_config_file, 
                                       videofile_path, 
                                       shuffle=shuffle, 
                                       videotype=VideoType, 
                                       c_engine=False)
    with output : 
        print('-- Detections and associations costs extracted')
    
def convert_tracklets():
    with terminal:
        deeplabcut.convert_detections2tracklets(path_config_file, Specific_videofile, videotype=VideoType,
                                                    shuffle=shuffle, track_method=tracktype, overwrite=True)
    with output : 
        print('-- Track body part assemblies frame-by-frame done')
        
def raw_2_tracks():
    # You need to point to your pickle file, please "copy path" from the folder to the left (right click, copy path)
    if tracktype == 'box':
        picklefile = 'Files/Inference/VideoDLC_resnet50_maDLC_MM_handsJul8shuffle1_20000_bx.pickle' #(see your video folder for path i.e. right click and say copy path!!!)
    elif tracktype == 'skeleton':
        picklefile = 'Files/Inference/VideoDLC_resnet50_maDLC_MM_handsJul8shuffle1_20000_sk.pickle' #(see your video folder for path i.e. right click and say copy path!!!)
    elif tracktype == 'ellipse':
        picklefile = 'Files/Inference/VideoDLC_resnet50_maDLC_MM_handsJul8shuffle1_20000_el.pickle' #(see your video folder for path i.e. right click and say copy path!!!)
    
    
    with terminal:
        deeplabcut.convert_raw_tracks_to_h5(path_config_file, picklefile)
        
    with output : 
        print('-- Raw to tracks done')
        
def filterpredictions():
    with terminal:
        deeplabcut.filterpredictions(path_config_file, 
                                 videofile_path, 
                                 videotype=VideoType, 
                                 track_method = tracktype)
    with output : 
        print('-- Predictions filtered')
        
def plot():
    with terminal:
        deeplabcut.plot_trajectories(path_config_file,videofile_path, videotype=VideoType, track_method=tracktype)
    with output : 
        print('-- Trajectories plotted')
        
def create():
    with terminal:
        deeplabcut.create_labeled_video(path_config_file,
                                videofile_path, 
                                shuffle=shuffle, 
                                draw_skeleton=False, 
                                videotype=VideoType, 
                                save_frames=False,
                                filtered=True, 
                                track_method = tracktype)
    with output : 
        print('-- Output video created')

# Full DLC methods
def dlc_full():
    dlcProgress.value+=1
    analyze_video()
    dlcProgress.value+=19
    convert_tracklets()
    dlcProgress.value+=20
    raw_2_tracks()
    dlcProgress.value+=15
    filterpredictions()
    dlcProgress.value+=15
    plot()
    dlcProgress.value+=15
    create()
    dlcProgress.value+=15
    
    # Free GPU memory - https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/discussion/96876 - i.e. Bazinga
    cuda.select_device(0)
    cuda.close()

In [9]:
# # --- Optional : For testing only if deeplabcut works well ---

# # VideoType = 'mp4'
# # path_config_file = '/home/ubuntu/Files/maDLC_MM_hands-Brice-2021-07-08/config.yaml'
# # videofile_path = ['/home/ubuntu/Files/Inference']

# print("Start Analyzing my video(s)!")
# scorername = deeplabcut.analyze_videos(path_config_file, 
#                                        videofile_path, 
#                                        shuffle=shuffle, 
#                                        videotype=VideoType, 
#                                        c_engine=False)

In [10]:
# --- 7.1 Data treatment for analysis ---
listF = []

def quantify():
    with terminal:
        # Step 1 : Read
        if tracktype == 'box':
            csv_path = '/home/ubuntu/Files/Inference/VideoDLC_resnet50_maDLC_MM_handsJul8shuffle1_20000_bx_filtered.csv'
        elif tracktype == 'skeleton':
            csv_path = '/home/ubuntu/Files/Inference/VideoDLC_resnet50_maDLC_MM_handsJul8shuffle1_20000_sk_filtered.csv'
        elif tracktype == 'ellipse':
            csv_path = '/home/ubuntu/Files/Inference/VideoDLC_resnet50_maDLC_MM_handsJul8shuffle1_20000_el_filtered.csv'
            
        video = pd.read_csv(csv_path)
        index = 0
        for column in video:
            video = video.rename(columns={column: index})
            index=index+1

        qProgress.value+=10
        # Step 2 : Keep only the interesting column and remove unused header row
        ## Keeps only interesting colums
        df = video[[0, 11, 20, 29, 38, 47, 59, 68, 77, 86, 95]]

        # Free python memory (!! Does not free computer memory !!)
        del video

        # Delete unused rows
        df = df.iloc[3:]

        qProgress.value+=10
        # Step 3 : Rename columns
        ## Rename columns
        df.columns = ['coords', 'LThumbEndY', 'LIndexEndY', 'LMiddleEndY', 'LRingEndY', 'LLittleEndY', 'RThumbEndY', 'RIndexEndY', 'RMiddleEndY', 'RRingEndY', 'RLittleEndY']
        
        with output:
            print("-- Left and right hands' fingers separated")

        # Step 4 : Normalize data and basic plots
        thumbs = extractData(df, 'LThumbEndY', 'RThumbEndY')
        indexs = extractData(df, 'LIndexEndY', 'RIndexEndY')
        middles = extractData(df,'LMiddleEndY', 'RMiddleEndY')
        rings = extractData(df,'LRingEndY', 'RRingEndY')
        littles = extractData(df, 'LLittleEndY', 'RLittleEndY')

        ## Create directory to save plots
        if not os.path.exists('plots'):
            os.mkdir('plots')

        ## Python3 code here creating class (https://www.geeksforgeeks.org/how-to-create-a-list-of-object-in-python-class/)
        class finger: 
            def __init__(self, df, name): 
                self.df = df 
                self.name = name

        listF.append( finger(thumbs, 'Thumbs') )
        listF.append( finger(indexs, 'Indexes') )
        listF.append( finger(middles, 'Middles') )
        listF.append( finger(rings, 'Rings') )
        listF.append( finger(littles, 'Littles') )
        qProgress.value+=10

        if not os.path.exists('plots'):
            os.mkdir('plots')
            
        if not os.path.exists('plots-normalized'):
            os.mkdir('plots-normalized')

        for obj in listF:
            customPlot(obj.df, obj.name)
            qProgress.value+=5

        for obj in listF:
            customPlotNormalized(obj.df, obj.name)
            qProgress.value+=5
            
        with output:
            print("-- Basic plots created and saved")


In [11]:
# --- 7.2 Methos for treamtents and normalization ---
# Helper methods
#Function to normalize a column
def normalize(col):
    min = col.min()
    max = col.max()
    return (col - min) / (max - min)

#Function to extract df from each end finger
def extractData(df, col1, col2):
    customData = df[['coords', col1, col2]]
    customData = customData.astype(float)
    customData = customData.dropna()

    #Normalize columns
    customData['LNorm'] = normalize(customData[col1])
    customData['RNorm'] = normalize(customData[col2])

    #Difference between L & R
    customData['L-R'] = (customData.LNorm - customData.RNorm).abs()

    return customData

#Function to show custom plot
def customPlot(df, name):
    plot = df.set_index('coords').plot(y=[0,1])
    plt.title(name)
    plt.ylabel('Pixels')
    plt.xlabel('Frames')
    plt.savefig('plots/'+name+'.png')
    plt.close()
    return plot

#Function to show custom plot normalized
def customPlotNormalized(df, name):
    plot = df.set_index('coords').plot(y=[2,3])
    plt.title(name + ' normalized')
    plt.ylabel('Pixels')
    plt.xlabel('Frames')
    name = name.replace(" ", "_")
    plt.savefig('plots-normalized/'+name+'-normalized.png')
    plt.close()
    return plot

In [12]:
# --- 8. Pearson correlation coefficient ---
def pearsonCorrelation(df):
    return round(df['LNorm'].corr(df['RNorm']),4)

def averageOfList(numOfList):
    avg = sum(numOfList) / len(numOfList)
    return avg

# for obj in list:
# #     print(pearsonCorrelation(obj.df))
#     plist.append(pearsonCorrelation(obj.df))
#   # pCorrelationMean = pCorrelationMean/5

def getPearsonCorrelation():
    plist = []    
    for obj in listF:
        plist.append(round(pearsonCorrelation(obj.df),4))    
    pCorrelationMean = averageOfList(plist) 
    return pCorrelationMean, plist

# pCorrelationMean = averageOfList(plist)


In [13]:
# --- 8. Cross correlation ---
def correlate(a, v, mode='valid', ddof=0, norm=True):
    """ (normalized) cross correlation """
    a = np.array(a)
    v = np.array(v)
    if len(a)<len(v):
        a,v = v,a
 
    if norm:
        a = a - a.mean() 
        v = v - v.mean()
 
    # zero padding of a, c is return vector
    if mode=='full': # return len(a) + len(v) -1 values
        c = np.empty(len(a)+len(v)-1)
        a = np.concatenate((np.zeros(len(v)-1), a, np.zeros(len(v)-1)))
    elif mode=='same': # return len(a) values
        c = np.empty(len(a))
        a = np.concatenate((np.zeros((len(v)-1)//2), a, np.zeros(len(v)-(len(v)-1)//2)))
    elif mode=='valid': # return len(a)-len(v)+1 values
        c = np.empty(len(a)-len(v)+1)
        pass
    else:
        print("error, do not know mode: {}".format(mode))
     
    # correlate
    for i in range(len(c)):
        #print (a.shape, v.shape, a[i:i+len(v)].shape, c.shape)
        c[i] = np.dot(a[i:i+len(v)], v)
        if norm:
            c[i] /= np.std(a[i:i+len(v)])*np.std(v)*(len(v)-ddof)
    return c

lineNr = 0
old = 0
def showXCorrelation (fing, name) :
    with terminal:
        op = correlate(fing.iloc[:, 3],fing.iloc[:, 4], mode = 'full')

        op = op[(int(len(op)/2))-10:int((len(op)/2))+10]

        plt.title('Normalized Cross-Correlation Time Lagged - '+name)
        #deze regel komt niet overeen met de maximal cross correleatie
        crossX = [item for item in range(len(op)) if op[item] == np.amax(op)]
        #range over 20 datapoints so -10 to get the correct datapoint of time-lag, multiplyed by time; 20ms
        plt.xlabel('CCCmax = ' + str(round(np.amax(op),2)) + '  Time = ' + str((crossX[0] - 10)*10))
        plt.axis([-10,10,np.amin(op),np.amax(op)])
        plt.grid()
        t = np.linspace((len(op)/2) * -1, len(op)/2, num = len(op))
        plt.plot(t, op, color='y')
        plt.savefig('plots-xcross/'+name+'_xcross.png')
        plt.close()
        
        #return the CCCmax
        return round(np.amax(op),2)

def showCrossCorrelation():
    #Create directory to save plots
    if not os.path.exists('plots-xcross'):
        os.mkdir('plots-xcross')
        
    maxCCCList = []

    for obj in listF:
        maxCCCList.append(showXCorrelation(obj.df, obj.name))
        qProgress.value+=4
        
    with output:
        print("Cross-correlation done")
        
    CCCmean = sum(maxCCCList) / len(maxCCCList)
        
    return CCCmean, maxCCCList


In [14]:
def full_quantification():
    quantify()
    CCCmean, CCClist = showCrossCorrelation()
    pCorrelationMean, plist = getPearsonCorrelation()
    with gb_1: 
        display(appendImages('plots'))
    with gb_2:
        display(appendImages('plots-normalized'))
    with gb_3:
        display(appendImages('plots-xcross'))
        
    text_pAvg= widgets.HTML(value=f'''<p>Pearson correlation average : {str(pCorrelationMean)}</p>''')
    text_pFingers = widgets.HTML(value=f'''<p>Pearson correlation for each finger from thumbs to little : {str(plist)}</p>''')
    text_WTscale= widgets.HTML(value=f'''<p>Level determined on Woods & Teuber scale for Pearson correlation: {str(determineLevel(pCorrelationMean))}</p>''')
    text_WTscale= widgets.HTML(value=f'''<p>Level determined on Woods & Teuber scale for Pearson correlation: {str(determineLevel(pCorrelationMean))}</p>''')
    with result_output:
        display(text_pAvg)
        display(text_pFingers)
        display(text_WTscale)
        
#         display(pCorrelationMean)
#         print('')
#         print('CCCmax average :' + str(CCCmean))
#         print('CCCmax for each finger from thumbs to little :' + str(CCClist))
#         print('Level determined on Woods & Teuber scale for CCCmax: ' + str(determineLevel(CCCmean)))
        
#         print('CCCmean')
#         display(CCCmean)
#         print('Level determined for CCCmax correlation: ' + str(determineLevel(CCCmean)))
#     text_download.layout.visibility='hidden'
    text_download.layout.display ='none'
              
def determineLevel(x):
    if(x>=0.96):
        return '4 = movement equal to that expected for the intended hand'
    elif(x>=0.82):
        return '3 = strong and sustained repetitive movement'
    elif(x>=0.66):
        return '2 = either slight but unsustained repetitive movement or stronger, but briefer, repetitive movement'
    elif(x>=0.50):
        return '1 = barely discernible repetitive movement'
    elif(x<0.50):
        return '0 = no clearly imitative movement'




    
# gb_1 = appendImages('plots')
# gb_2 = appendImages('plots-normalized')
# gb_3 = appendImages('plots-xcross')
# accordion = widgets.Accordion(description='Plots', children=[gb_1])
# vbox_center = widgets.VBox([hbox_tool, accordion, accordion2, accordion3, accordion4], layout=Layout(width="55%")) #, justify_content='center', align_self='center', align_items='center'))
# hbox_body = widgets.HBox([vbox_status, vbox_center, vbox_instructions], layout=Layout(display="flex", width="100%", justify_content='space-between'))




# # # --- Optional : See the layout ---
# page = widgets.VBox([hbox_header, hbox_body, hbox_footer])
# # display(page)

# with output:
#     display(gb_1)

In [39]:
# # text_download.layout.display ='none'
# text_download.layout.display = "inline-block"
# # text_download.layout.display = "block"
# # text_download.layout.display = "inline"

# # --- 5. Assign layout ---
# page = widgets.VBox([hbox_header, hbox_body, hbox_footer])
# display(page)

